Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2337, done.
remote: Total 2337 (delta 0), reused 0 (delta 0), pack-reused 2337
Receiving objects: 100% (2337/2337), 8.09 MiB | 39.45 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.


In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 686kB 24.4MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp37-none-any.whl size=655251 sha256=fc3de98e5580c4e619a3050ee605d4df83e2872b04fe240c7af914c466322b1e
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=c8828b1b512fb43f4a48a1823592a91502b0e3e3fba04bf34ec85854abf1ce9e
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile


# Datasets

-   Create a dataset folder under `/dataset` for your dataset.
-   Create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. Place any images you want to transform from a to b (cat2dog) in the `testA` folder, images you want to transform from b to a (dog2cat) in the `testB` folder, and do the same for the `trainA` and `trainB` folders.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [5]:
# --continue_train --epoch_count 100 
# --batch_size 4 (?)
!python train.py --continue_train --epoch_count 5 --dataroot /content/drive/MyDrive/dataset --name sim2real_big --model cycle_gan --checkpoints_dir /content/drive/MyDrive/mycheckpoints --batch_size 4 --n_epochs 40 --n_epochs_decay 10 --display_winsize 128 --ngf 32 --ndf 32 --save_by_iter 

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc8872884d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1297, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 921, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.7/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.p

In [16]:
# Using latest A -> B 'converter'
!cp /content/drive/MyDrive/mycheckpoints/sim2real/latest_net_G_A.pth /content/drive/MyDrive/mycheckpoints/sim2real/latest_net_G.pth

# Using latest B -> A 'converter'
#!cp /content/drive/MyDrive/mycheckpoints/sim2real/latest_net_G_B.pth /content/drive/MyDrive/mycheckpoints/sim2real/latest_net_G.pth

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [24]:
!rm -r /content/pytorch-CycleGAN-and-pix2pix/results/

In [25]:
# Testing latest A -> B 'converter'
!python test.py --dataroot /content/drive/MyDrive/dataset/testA --name sim2real --model test --no_dropout --checkpoints_dir /content/drive/MyDrive/mycheckpoints --num_test 10000

# Testing latest B -> A 'converter'
#!python test.py --dataroot /content/drive/MyDrive/dataset/testB --name sim2real --model test --no_dropout --checkpoints_dir /content/drive/MyDrive/mycheckpoints --num_test 10000

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/drive/MyDrive/mycheckpoints	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/dataset/testA	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                loa

# Downloading results

In [26]:
!zip -r /content/results.zip /content/pytorch-CycleGAN-and-pix2pix/results

Output streaming troncato alle ultime 5000 righe.
  adding: content/pytorch-CycleGAN-and-pix2pix/results/sim2real/test_latest/images/2848_cam-image_array__real.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/sim2real/test_latest/images/1624_cam-image_array__real.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/sim2real/test_latest/images/2512_cam-image_array__real.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/sim2real/test_latest/images/671_cam-image_array__fake.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/sim2real/test_latest/images/1921_cam-image_array__real.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/sim2real/test_latest/images/2812_cam-image_array__real.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/sim2real/test_latest/images/1521_cam-image_array__fake.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/

In [28]:
from google.colab import files
files.download('/content/results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>